In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnull , when , count, regexp_replace , udf
from pyspark.sql.types import StringType
import re


In [8]:
# Initialize a Spark session
spark = SparkSession.builder \
        .appName("PySpark Test") \
        .getOrCreate()


    
    
        
df_customers = spark.read.csv("CRM_data/customers.csv" , header = True , inferSchema = True)
df_interactions = spark.read.csv("CRM_data/interactions.csv" , header = True , inferSchema = True)
df_products = spark.read.csv("CRM_data/products.csv" , header = True , inferSchema = True)
df_sales = spark.read.csv("CRM_data/sales_team.csv" , header = True , inferSchema = True)
df_transactions = spark.read.csv("CRM_data/transactions.csv" , header = True , inferSchema = True)
    
    # data accuracy checks
    
      # customer_id checks
    
        
    
df_customers.createOrReplaceTempView("customers_table")
df_transactions.createOrReplaceTempView("transactions_table")
df_interactions.createOrReplaceTempView("interactions_table")
df_products.createOrReplaceTempView("products_table")
df_sales.createOrReplaceTempView("sales_table")
    
    # data accuracy of customers and transactions files
    
    
    


  

        

In [9]:
df_common = df_transactions.join(df_customers, on = "Customer_ID" , how = 'inner')
df_common.show()



+--------------------+--------------------+----------+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|         Customer_ID|      Transaction_ID|Product_ID|      Date|Amount|        Sales_Rep_ID|                Name|               Email|               Phone|             Country|
+--------------------+--------------------+----------+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|d7522cab-f713-4c7...|31190b6c-54cd-4cd...|        48|01-04-2024|   234|8aede4f3-bd43-45a...|         Sarah Duran|francisleblanc@ex...|001-863-315-0524x...|Lao People's Demo...|
|dae0689d-0c38-440...|05d4bcca-dc8e-405...|         3|28-03-2024|   119|c48b42a3-544b-454...|      Margaret Terry| jmiller@example.net|        516-680-4883|             Bahrain|
|075005e4-bb34-496...|9b6e01be-1814-41e...|        26|09-06-2024|   240|d7558c8d-5355-48e...|       David Mcma

In [10]:
#data accuracy in customers and transaction tables
df_not_in_costumers_transactions = df_transactions.join(df_customers, on='Customer_ID', how = "left_anti")
df_not_in_costumers_transactions.show()

+-----------+--------------+----------+----+------+------------+
|Customer_ID|Transaction_ID|Product_ID|Date|Amount|Sales_Rep_ID|
+-----------+--------------+----------+----+------+------------+
+-----------+--------------+----------+----+------+------------+



In [11]:
#data accuracy in customers and interactions tables
df_not_in_customers_interactions = df_interactions.join(df_customers, on='Customer_ID',how="left_anti")
df_not_in_customers_interactions.show()

+-----------+--------------+----------------+----------------+--------------+
|Customer_ID|Interaction_ID|Interaction_Date|Interaction_Type|Issue_Resolved|
+-----------+--------------+----------------+----------------+--------------+
+-----------+--------------+----------------+----------------+--------------+



In [12]:
#data accuracy in products and transactions
df_not_in_products_transactions = df_transactions.join(df_products, on='Product_ID',how = "left_anti")
df_not_in_products_transactions.show()

+----------+--------------+-----------+----+------+------------+
|Product_ID|Transaction_ID|Customer_ID|Date|Amount|Sales_Rep_ID|
+----------+--------------+-----------+----+------+------------+
+----------+--------------+-----------+----+------+------------+



In [13]:
#data accuracy in sales and transactions team
df_not_in_sales_transactions = df_transactions.join(df_sales,on='Sales_Rep_ID',how='left_anti')
df_not_in_sales_transactions.show()

+------------+--------------+-----------+----------+----+------+
|Sales_Rep_ID|Transaction_ID|Customer_ID|Product_ID|Date|Amount|
+------------+--------------+-----------+----------+----+------+
+------------+--------------+-----------+----------+----+------+



In [14]:
#data accuracy in sales and transactions team
df_not_in_sales_transactions = df_transactions.join(df_sales,on='Sales_Rep_ID',how='left_anti')
df_not_in_sales_transactions.show()

+------------+--------------+-----------+----------+----+------+
|Sales_Rep_ID|Transaction_ID|Customer_ID|Product_ID|Date|Amount|
+------------+--------------+-----------+----------+----+------+
+------------+--------------+-----------+----------+----+------+



In [15]:
df_sales.describe().show()

24/08/22 11:06:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+---------------+---------+------------------+------------------+
|summary|        Sales_Rep_ID|           Name|   Region|      Sales_Target|    Sales_Achieved|
+-------+--------------------+---------------+---------+------------------+------------------+
|  count|                  52|             52|       52|                52|                47|
|   mean|                NULL|           NULL|     NULL| 27089.55769230769|21912.425531914894|
| stddev|                NULL|           NULL|     NULL|11526.293189177553| 9799.237937860891|
|    min|02acdbab-d149-405...|Alicia Johnston|  Alabama|             10385|              6119|
|    max|f243144e-485f-438...|    William Lee|Wisconsin|             49186|             38252|
+-------+--------------------+---------------+---------+------------------+------------------+



In [16]:
df_customers.dropDuplicates()
df_customers.show()

+--------------------+-----------------+--------------------+--------------------+--------------------+
|         Customer_ID|             Name|               Email|               Phone|             Country|
+--------------------+-----------------+--------------------+--------------------+--------------------+
|a85e6a90-78d5-490...|   Shannon Deleon|                NULL|          5878628895|               Japan|
|babec972-ffb3-4c5...|Christina Sanchez|craigprice@exampl...|          4832368495|               Haiti|
|d74c33bd-69d9-471...|     Thomas Brown|vjohnson@example.org|       (276)903-7065|            Pakistan|
|ff05ceba-f459-471...| Lindsey Bradford|kathryn50@example...|                NULL|    Marshall Islands|
|f20755f6-8481-490...|       John Boyer|jennifer15@exampl...|       (749)644-5721|       New Caledonia|
|2140af6b-0b34-434...|       Paula Sims|angelalong@exampl...|  695-406-8297x62008|             Vietnam|
|e390fd9b-f416-417...|      Erin Porter|sharonwood@exampl...|001

In [17]:
df_transactions.show()
df_transactions.dropDuplicates()
df_transactions.show()

+--------------------+--------------------+----------+----------+------+--------------------+
|      Transaction_ID|         Customer_ID|Product_ID|      Date|Amount|        Sales_Rep_ID|
+--------------------+--------------------+----------+----------+------+--------------------+
|31190b6c-54cd-4cd...|d7522cab-f713-4c7...|        48|01-04-2024|   234|8aede4f3-bd43-45a...|
|05d4bcca-dc8e-405...|dae0689d-0c38-440...|         3|28-03-2024|   119|c48b42a3-544b-454...|
|9b6e01be-1814-41e...|075005e4-bb34-496...|        26|09-06-2024|   240|d7558c8d-5355-48e...|
|46bd8f7c-20e0-405...|ec910504-7738-463...|        37|24-03-2024|   451|d0fa26a0-d161-497...|
|f02dded4-81bf-431...|84441f52-77e5-450...|         4|29-05-2024|    78|8aede4f3-bd43-45a...|
|9b068943-af5f-4e4...|2b2673a4-7eed-477...|        20|23-07-2024|   382|5f719abf-9d76-49d...|
|810a7e7a-8a84-4f0...|742f0811-2251-480...|        16|10-06-2024|   191|d0fa26a0-d161-497...|
|a62a6e9d-6520-457...|c7f17f22-4ba9-4d2...|        12|07-04-

In [18]:
df_interactions.dropDuplicates()
df_products.dropDuplicates()
df_sales.dropDuplicates()

DataFrame[Sales_Rep_ID: string, Name: string, Region: string, Sales_Target: int, Sales_Achieved: int]

In [19]:
#data cleaning and transformation as a part of cleaning
#customer dataframe - column having anamolies - 'Phone'

df_cleaned_data = df_customers.withColumn('Phone' , regexp_replace('Phone', r'[()-.x]','') )
df_cleaned_data.show()
    


+--------------------+-----------------+--------------------+-----------------+--------------------+
|         Customer_ID|             Name|               Email|            Phone|             Country|
+--------------------+-----------------+--------------------+-----------------+--------------------+
|a85e6a90-78d5-490...|   Shannon Deleon|                NULL|       5878628895|               Japan|
|babec972-ffb3-4c5...|Christina Sanchez|craigprice@exampl...|       4832368495|               Haiti|
|d74c33bd-69d9-471...|     Thomas Brown|vjohnson@example.org|       2769037065|            Pakistan|
|ff05ceba-f459-471...| Lindsey Bradford|kathryn50@example...|             NULL|    Marshall Islands|
|f20755f6-8481-490...|       John Boyer|jennifer15@exampl...|       7496445721|       New Caledonia|
|2140af6b-0b34-434...|       Paula Sims|angelalong@exampl...|  695406829762008|             Vietnam|
|e390fd9b-f416-417...|      Erin Porter|sharonwood@exampl...|00146327608206208| Trinidad an

In [20]:
df_customers = df_cleaned_data
df_customers.show()



+--------------------+-----------------+--------------------+-----------------+--------------------+
|         Customer_ID|             Name|               Email|            Phone|             Country|
+--------------------+-----------------+--------------------+-----------------+--------------------+
|a85e6a90-78d5-490...|   Shannon Deleon|                NULL|       5878628895|               Japan|
|babec972-ffb3-4c5...|Christina Sanchez|craigprice@exampl...|       4832368495|               Haiti|
|d74c33bd-69d9-471...|     Thomas Brown|vjohnson@example.org|       2769037065|            Pakistan|
|ff05ceba-f459-471...| Lindsey Bradford|kathryn50@example...|             NULL|    Marshall Islands|
|f20755f6-8481-490...|       John Boyer|jennifer15@exampl...|       7496445721|       New Caledonia|
|2140af6b-0b34-434...|       Paula Sims|angelalong@exampl...|  695406829762008|             Vietnam|
|e390fd9b-f416-417...|      Erin Porter|sharonwood@exampl...|00146327608206208| Trinidad an

In [23]:
df_afghanistan = df_customers.filter(df_customers['country'] == 'Afghanistan')
df_afghanistan.show()

+--------------------+----------+--------------------+-----+-----------+
|         Customer_ID|      Name|               Email|Phone|    Country|
+--------------------+----------+--------------------+-----+-----------+
|61a98216-3f7a-4a9...|Emily Reed|stephenrich@examp...| 8382|Afghanistan|
+--------------------+----------+--------------------+-----+-----------+



In [24]:


def clean_phone_number(Phone):
    # Remove unwanted characters
    Phone = re.sub(r'[\(\)\.\-\s]', '', Phone)
    # Remove extensions
    Phone = re.sub(r'x\d+', '', Phone)
    # Handle country code if needed
    if not Phone.startswith('+'):
        Phone = '+1-' + Phone
    # Format the number
    if len(Phone) == 11:
        Phone = Phone[:3] + '-' + Phone[3:6] + '-' + Phone[6:]
    return Phone


clean_phone_udf = udf(clean_phone_number, StringType())

# Apply the UDF to the DataFrame
df_cleaned = df_customers.withColumn("cleaned_phone", clean_phone_udf(df_customers["Phone"]))


NameError: name 'udf' is not defined

24/08/23 03:13:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 31022287 ms exceeds timeout 120000 ms
24/08/23 03:13:22 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/23 03:13:22 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint

In [22]:


# if __name__ == "__main__":
#     main()

    